# Depresión en México

## Instalación de Librerías

In [1]:
# pip install twint
# pip install nltk

## Importación de Librerías

In [2]:
import pandas as pd
import numpy as np
import nltk

import twint
import nest_asyncio
from os import remove

## Funciones

In [3]:
def deleteDownloadedData(data, path):
    for index in range(0, data.shape[0]):
        remove("{}/{}.csv".format(path, data["state"].iloc[index]))

In [4]:
def concatenateData(data, path):
    twitter = pd.read_csv("{}/{}.csv".format(path, data["state"].iloc[0]), index_col=0)
    for index in range(1, data.shape[0]):
        tweets = pd.read_csv("{}/{}.csv".format(path, data["state"].iloc[index]), index_col=0)
        twitter = pd.concat([twitter, tweets])
    twitter.to_csv("{}/twitter-raw.csv".format(path), index=False)
    deleteDownloadedData(data, path)

In [5]:
def downloadTwitterData(data, path, kilometers=10, limit=16000):
    for index in range(0, data.shape[0]):

        state, longitude, latitude = data.iloc[index]

        c = twint.Config()
        c.Geo = f"{longitude}, {latitude}, {kilometers}km"
        c.Limit = limit
        c.Store_csv = True
        c.Output = f"{path}/{state}.csv"
        c.Lang = "es"
        c.Hide_output = True
    
        nest_asyncio.apply()
        twint.run.Search(c)
        nest_asyncio.apply(loop=None)

    concatenateData(data, path)

## Directorio de Datos

Es necesario crear una variable que almacene la ruta de acceso relativa en la que se encuentran los datos.

In [6]:
path = "./data"

## Recolección de Información

Para comenzar, vamos a descargar un determinado número de publicaciones de Twitter por cada estado de la Republica Mexicana.

Este proceso se realiza mediante la función <code>downloadTwitterData</code>.

In [7]:
# downloadTwitterData(pd.read_csv("{}/mexico-states.csv".format(path)), path, 20, 16000)

## Pre-procesamiento del Conjunto de Datos

Una vez recolectada la información, es necesario llevar a cabo una limpieza sobre los datos crudos para conservar unicamente la información con mayor utilidad.

### Lectura del Conjunto de Datos

Este proceso se realiza mediante la función <code>read_csv</code> de la librería <code>pandas</code>.

In [8]:
twitter_raw = pd.read_csv("{}/twitter-raw.csv".format(path))

### Exploración

Una primer aproximación es visualizar una pequeña muestra del conjunto de datos.

Este proceso se realiza mediante la función <code>head</code> de la librería <code>pandas</code>.

In [9]:
twitter_raw.head()

,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1402175394787303425,2021-06-08 03:07:19 Hora de verano central (Mé...,2021-06-08,03:07:19,-500,875856864,marianamshorago,Shor,NaN,🤷🏼‍♀️,...,"21.8853,-102.2916,10km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1402074003293061121,2021-06-08 03:06:45 Hora de verano central (Mé...,2021-06-08,03:06:45,-500,926552430648745984,abnerlop83,Abnerlop83🍓SLC🤘,NaN,@AngelaAguilar__ vamos a llorar con esta canci...,...,"21.8853,-102.2916,10km",NaN,NaN,NaN,NaN,"[{'screen_name': 'AngelaAguilar__', 'name': 'Á...",NaN,NaN,NaN,NaN
2,1402174384958234628,2021-06-08 03:06:26 Hora de verano central (Mé...,2021-06-08,03:06:26,-500,2319584149,fcoo_diaz,Francisco Dávila Díaz,NaN,@HyDro_Licks Love you 🥰,...,"21.8853,-102.2916,10km",NaN,NaN,NaN,NaN,"[{'screen_name': 'HyDro_Licks', 'name': 'Dro T...",NaN,NaN,NaN,NaN
3,1402174637019074569,2021-06-08 03:04:19 Hora de verano central (Mé...,2021-06-08,03:04:19,-500,2866094780,princessxpita,P 🐰,NaN,i might cry a lil,...,"21.8853,-102.2916,10km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1401452915131301890,2021-06-08 03:04:09 Hora de verano central (Mé...,2021-06-08,03:04:09,-500,1168310701095641088,juliogomro,someone finally,NaN,@Yesbi16 @sofilozanno Se arma,...,"21.8853,-102.2916,10km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Yesbi16', 'name': 'Yesbi', '...",NaN,NaN,NaN,NaN


Como podemos observar, cada registro contiene información detallada acerca de una publicación, para ser exactos, cada registro está compuesto por 35 columnas.

### Selección de Columnas

Después de un breve análisis, determinamos que las columnas más relevantes son: <code>user_id</code>, <code>date</code>, <code>tweet</code>, <code>language</code>.

Así que creamos un nuevo <code>DataFrame</code> que contenga unicamente las columnas más relevantes.

In [10]:
twitter_data = twitter_raw[["user_id", "date", "tweet", "language"]]

### Subtitle Lv.3

Es necesario asegurar que los _metadatos_ corresponden adecuadamente a la información que almacena cada columna de nuestro conjunto de datos.

Este proceso se realiza mediante la función <code>dtypes</code> de la librería <code>pandas</code>.

In [11]:
twitter_data.dtypes

user_id      int64
date        object
tweet       object
language    object
dtype: object

Como podemos observar, el tipo de dato de la columna <code>date</code> no corresponde a la información que almacena, por lo que es necesaria una conversión de <code>Object</code> a <code>datetime64</code>.

Este proceso se realiza mediante la función <code>astype</code> de la librería <code>pandas</code>.

In [12]:
twitter_data = twitter_data.astype({"date": "datetime64[ns]"})

### Selección de Registros

Dado que los registros contienen publicaciones hechas en distintos idiomas, es necesario conservar unicamente las publicaciones en español y además, descartar la columna <code>language</code>.

In [13]:
twitter_data = twitter_data[twitter_data["language"] == "es"][["user_id", "date", "tweet"]]

### Respaldo

Finalmente creamos un respaldo del conjunto de datos procesado previamente.

Este proceso se realiza mediante la función <code>to_csv</code> de la librería <code>pandas</code>.

In [15]:
twitter_data.to_csv("{}/twitter.csv".format(path), index=False, encoding="utf-8-sig")

## Subtitle Lv.2